# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [20]:
from openai import OpenAI

client = OpenAI(base_url="https://api.xiaoai.plus/v1",api_key="sk-tdqfro61NniG2LZd5394E5F28c51419d8b4dE7Db5a6105C4")

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图描绘了一个宁静的自然景观。图中显示一条木质栈道穿过一片郁郁葱葱的草地，直通远处的一片树林和疏松的丛林。草地呈现出生机勃勃的绿色，而天空是广阔的，布满了细腻的白云。光线从云层中透出，为整个景象增添了一抹温暖的色彩。这样的场景可能位于一个自然保护区或者是一个公园，是一个适合散步或者亲近自然的地方。整体上，这幅图展示了大自然的宁静与美丽。', role='assistant', function_call=None, tool_calls=None))


In [21]:
response.choices[0].message.content

'这幅图描绘了一个宁静的自然景观。图中显示一条木质栈道穿过一片郁郁葱葱的草地，直通远处的一片树林和疏松的丛林。草地呈现出生机勃勃的绿色，而天空是广阔的，布满了细腻的白云。光线从云层中透出，为整个景象增添了一抹温暖的色彩。这样的场景可能位于一个自然保护区或者是一个公园，是一个适合散步或者亲近自然的地方。整体上，这幅图展示了大自然的宁静与美丽。'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默风格的搞笑比较。图中展示了两种不同状态的狗。左边是一只被幻想成拥有非常发达肌肉、类似人类健美运动员身体的柴犬，配文“16岁的我，工作后的我”，表达了年轻时充满活力和梦想的状态。右边的柴犬则显得普通、略显憔悴，配有文字“好累好困、好想摸鱼、看到鸡腿就开心，不属于我、我也有小肚腩、肉肉的很可爱”，描绘了工作后可能感到疲惫、减少锻炼而略显发福的现实状态。

这种对比通常用来幽默地表达人们对于理想与现实之间差距的感慨，展示了许多人对于年轻时的憧


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [2]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(base_url="https://api.xiaoai.plus/v1",api_key="sk-tdqfro61NniG2LZd5394E5F28c51419d8b4dE7Db5a6105C4")
def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [23]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图表展示了1980年至2020年期间美国、中国、日本和德国的GDP（国内生产总值）对比。横轴表示年份，从1980年到2020年，纵轴表示GDP值，单位是万亿美元。

从图表可以看到：

1. 美国（蓝线）的GDP呈上升趋势，尤其在2000年后增长较快，到2020年达到最高点，约20万亿美元。
2. 中国（红线）的GDP增长非常显着，特别是从2000年之后，其增长速度明显加快，到2020年接近美国，约为15万亿美元。
3. 日本（紫线）的GDP在1995年左右达到顶峰后，有所波动，但总体趋势稳定，在5万亿美元左右。
4. 德国（绿线）的GDP相对平稳，增长幅度较小，基本在3至4万亿美元之间波动。

总体来看，这张图表显示了过去四十年中四个国家的经济增长情况，其中美国和中国显示了较大的增长动力。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [24]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片是一段关于深度学习中特定技术的笔记内容，主要讨论了“Prompt Tuning”和“LoRA”（Low-Rank Adaptation）等技术。

1. **Prompt Tuning (面向小模型的方法)**：这里记录了Prompt Tuning的基本概念，即把输入X通过转换矩阵W映射到输出Y，X是输入的Embedding表示，可以是多个Token的Embedding向量。这种方法常用于参数数量较少的模型调优。

2. **Prefix Tuning**：这部分描述了一种仅调整输入序列前缀的调优方式，通过设计特定的前缀词向量来影响模型的输出。

3. **LoRA (低秩调整)**：这部分介绍了LoRA技术，用于在不增加过多计算复杂度和参数数量的情况下调整深度神经网络。LoRA通过引入一个低秩矩阵ΔW来修改原有的权重矩阵W，从而实现模型的微调。文中提到了使用矩阵分解技术(ΔW=AB)来减少参数数量，这有助于保持模型的高效和可管理。

4. 文末提到了两个数据点，显示LAMA和QLoRA的参数大小，LoRA为78GB，而QLoRA为48GB，表明通过这种技术可以有效减小模型的大小。

整体来看，这些笔记概述了几种在不同设置下对深度神经网络进行微调的技术，特别是在保持模型大小和复杂度可控的同时调整模型性能的方法。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [4]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片展示的是一个笔记本上手写的中文笔记。从图片中可以看到笔记内容涉及多个方面，包括个人学习计划、提醒和目标。部分内容涉及特定的学习任务，比如MOOC（大规模开放在线课程）的学习计划、阅读文章以及追踪学习进展。还有关于API学习的计划和一些强调重点。这些笔记显得有系统而且条理清晰，反映出记录者对其学习和工作内容的认真态度。

![](./images/handwriting_1.jpg)

In [26]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片是一本笔记本的两页内容，记录了一些关于机器学习、特别是自然语言处理领域中的技术和方法的笔记。

左页的标题是“自然语言处理”，下面列出了若干与Transformers模型相关的术语和概念，如PEFT (可能是指某种Fine-Tuning技术)，SOTA (State of the Art，技术中的最高标准或成就)，PBFT Methods等。整页笔记看起来是在讨论不同的训练技术和模型架构。

右页的内容涉及了更多高级的机器学习概念，如multi-modality，LLMs (Large Language Models)，以及各种Fine-Tuning的技术，例如Adapter, Prefix Tuning, 和LoRA。这些都是用于改进模型性能或适应特定任务的技术。还提到了一些模型名称，如GPT、BERT等。

笔记用中英双语写成，显示了笔者在整理自己的学习或研究成果时使用的思考和记录方式。整体而言，这些笔记很可能是用于学术研究或深入学习自然语言处理技术的个人笔记。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [7]:
#使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。
content = query_base64_image_description("./images/note.png")
display(Markdown(content))

这张图片显示的是一张纸上写的一些笔记信息，具体内容如下：

标题是“孩子标准”，下面列举了几个孩子标准的要求：
1. 体质指数 (BMI) 的标准体脂率为：
   - 男孩：0.7倍、正常、1.5倍、注意肥胖
   - 女孩：0.7倍、正常、1.5倍、注意肥胖
2. 国际贸易威胁方面的内容（但该内容以缩写形式出现不完整）
3. 还提到了“备胎"。

这看起来像是一些生理健康指标的标准，以及一些可能与社会科学或经济相关的要点。具体内容可能需要在更广泛的背景或详细信息中才能完全理解。

In [8]:
#整合 query_base64_image_description 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。
def display_image_description(image_path):
    description = query_base64_image_description(image_path)
    display(Markdown(description))
display_image_description("./images/note.png")

这张图片显示的是一段用手写体书写的中文笔记，内容涉及数学概率的题目。具体内容如下：

题目标题是“孙红旗”，指的可能是出题人或相关知识点。

题目内容：
0. 水中(pi)含有隔绝菌家族：
①浓度 0.7倍↓、低配、1.5倍↓、汹涌恶劣
②国际贸易影响各地质的质问题意(不)
④ 答案可能错。

这段内容首先提到了水中的一种成分pi含有某种细菌的浓度情况，并用一些难以理解的语句描述了这一点。接着用一句可能有误或不完整的表述指出国际贸易影响了某种情况。最后提及可能的答案错误。整体上，这段内容在专业性和表述上显得有些模糊和难以直接理解，可能需要具体的学科背景知识或上下文信息才更加清楚具体是什么意思。